In [1]:
cd ..

/home/dmitriishubin/Desktop/Thesis/HepaticVessel_experiment


In [2]:
import pandas as pd
import numpy as np
import pydicom as dicom
import nibabel as nib
import os
from sklearn.model_selection import KFold,train_test_split
import json

/home/dmitriishubin/anaconda3/envs/thesis/lib/python3.7/site-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


In [3]:
img = nib.load('../data/Task05_Prostate/imagesTr/prostate_00.nii.gz.')
img = np.array(img.dataobj)

In [4]:
img.shape

(512, 512, 304)

In [5]:
patients = [i for i in os.listdir('../data/Task05_Prostate/imagesTr/') if i.find('._pr')==-1]

In [6]:
patients

['prostate_46.nii.gz',
 'prostate_01.nii.gz',
 'prostate_13.nii.gz',
 'prostate_42.nii.gz',
 'prostate_39.nii.gz',
 'prostate_16.nii.gz',
 'prostate_32.nii.gz',
 'prostate_40.nii.gz',
 'prostate_25.nii.gz',
 'prostate_44.nii.gz',
 'prostate_02.nii.gz',
 'prostate_17.nii.gz',
 'prostate_28.nii.gz',
 'prostate_34.nii.gz',
 'prostate_04.nii.gz',
 'prostate_31.nii.gz',
 'prostate_21.nii.gz',
 'prostate_43.nii.gz',
 'prostate_29.nii.gz',
 'prostate_38.nii.gz',
 'prostate_37.nii.gz',
 'prostate_00.nii.gz',
 'prostate_07.nii.gz',
 'prostate_35.nii.gz',
 'prostate_41.nii.gz',
 'prostate_24.nii.gz',
 'prostate_10.nii.gz',
 'prostate_20.nii.gz',
 'prostate_47.nii.gz',
 'prostate_06.nii.gz',
 'prostate_14.nii.gz',
 'prostate_18.nii.gz']

In [9]:
patients[0][:-7]

'lung_054'

In [28]:
names = []

os.makedirs('../data/Task06_Lung/processed_data',exist_ok=True)

count = 0

for patient in patients:
    img = nib.load(f'../data/Task06_Lung/imagesTr/{patient}')
    img = np.array(img.dataobj)
    seg = nib.load(f'../data/Task06_Lung/labelsTr/{patient}')
    seg = np.array(seg.dataobj)
    
    patient = patient[:-7]
    
    os.makedirs(f'../data/Task06_Lung/processed_data/{patient}',exist_ok=True)
    
    for i in range(img.shape[-1]):
        temp_img = img[:,:,i].reshape(320, 320,1)
        temp_seg = seg[:,:,i].reshape(320, 320,1)
        
        
        np.save(f'../data/Task06_Lung/processed_data/{patient}/{i}.npy',temp_img)
        np.save(f'../data/Task06_Lung/processed_data/{patient}/{i}_seg.npy',temp_seg)
        count+=1
            

In [7]:
for index,patient in enumerate(patients):
    patients[index]=patient[:-7]

In [13]:
patients = np.array(patients)

kf = KFold(5,shuffle=True,random_state=42)
for fold,(train_index, test_index) in enumerate(kf.split(patients)):
    patients_train, patients_test = patients[train_index], patients[test_index]
    
    images_train = []
    for patient in patients_train:
        images_train+=[
                f'../data/Task06_Lung/processed_data/{patient}/'+i[:-4]
                for i in sorted(os.listdir(f'../data/Task09_Spleen/processed_data/{patient}/'))
                if i.find('.npy') != -1 and i.find('seg') == -1
            ]
    
    
    images_val = []
    for patient in patients_test:
        images_val+=[
                f'../data/Task06_Lung/processed_data/{patient}/'+i[:-4]
                for i in sorted(os.listdir(f'../data/Task09_Spleen/processed_data/{patient}/'))
                if i.find('.npy') != -1 and i.find('seg') == -1
            ]
        
    
    split = {
        'train': images_train,
        'val': images_val,
    }
    
    
    with open(f'../data/split_table/{fold}_split_table.json', 'w') as outfile:
            json.dump(split, outfile)

In [16]:
patients

['lung_054',
 'lung_047',
 'lung_014',
 'lung_093',
 'lung_042',
 'lung_095',
 'lung_006',
 'lung_037',
 'lung_023',
 'lung_058',
 'lung_055',
 'lung_065',
 'lung_022',
 'lung_066',
 'lung_073',
 'lung_071',
 'lung_031',
 'lung_086',
 'lung_053',
 'lung_025',
 'lung_048',
 'lung_096',
 'lung_003',
 'lung_043',
 'lung_079',
 'lung_045',
 'lung_036',
 'lung_028',
 'lung_016',
 'lung_041',
 'lung_062',
 'lung_070',
 'lung_075',
 'lung_018',
 'lung_059',
 'lung_004',
 'lung_069',
 'lung_057',
 'lung_064',
 'lung_001',
 'lung_015',
 'lung_027',
 'lung_081',
 'lung_005',
 'lung_049',
 'lung_051',
 'lung_061',
 'lung_092',
 'lung_078',
 'lung_083',
 'lung_038',
 'lung_026',
 'lung_020',
 'lung_009',
 'lung_029',
 'lung_010',
 'lung_034',
 'lung_080',
 'lung_033',
 'lung_074',
 'lung_046',
 'lung_044',
 'lung_084']

In [8]:
patients = np.array(patients)

for fold in range(5):

    patients_train, patients_test = train_test_split(patients, test_size=0.2, random_state=fold)

    patients_train, patients_val = train_test_split(patients_train, test_size=0.25, random_state=fold)

    images_train = []
    for patient in patients_train:
        images_train+=[
                f'../data/Task05_Prostate/processed_data/{patient}/'+i[:-4]
                for i in sorted(os.listdir(f'../data/Task05_Prostate/processed_data/{patient}/'))
                if i.find('.npy') != -1 and i.find('seg') == -1
            ]


    images_val = []
    for patient in patients_val:
        images_val+=[
                f'../data/Task05_Prostate/processed_data/{patient}/'+i[:-4]
                for i in sorted(os.listdir(f'../data/Task05_Prostate/processed_data/{patient}/'))
                if i.find('.npy') != -1 and i.find('seg') == -1
            ]

    images_test = []
    for patient in patients_test:
        images_test+=[
                f'../data/Task05_Prostate/processed_data/{patient}/'+i[:-4]
                for i in sorted(os.listdir(f'../data/Task05_Prostate/processed_data/{patient}/'))
                if i.find('.npy') != -1 and i.find('seg') == -1
            ]


    split = {
        'train': images_train,
        'val': images_val,
        'test': images_test,
    }


    with open(f'../data/split_table/{fold}_split_table.json', 'w') as outfile:
            json.dump(split, outfile)

In [9]:
len(patients_train)

18

In [10]:
len(patients_val)

7

In [19]:
f'../data/Task02_Heart/processed_data/{patients_train}/'

"../data/Task02_Heart/processed_data/['la_007' 'la_019' 'la_009' 'la_016' 'la_004' 'la_022' 'la_018' 'la_014'\n 'la_011' 'la_010' 'la_024' 'la_020' 'la_023' 'la_030' 'la_021' 'la_017']/"